In [1]:
import pandas as pd
import numpy as np

imbd_reviews=pd.read_csv('IMDB Dataset.csv')

In [2]:
imbd_reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
imbd_reviews["review"] = imbd_reviews['review'].str.replace('[^\w\s]','')

C:\Users\margen\AppData\Local\Temp\ipykernel_10196\1400499348.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  imbd_reviews["review"] = imbd_reviews['review'].str.replace('[^\w\s]','')


In [4]:
imbd_reviews["review"]

0        One of the other reviewers has mentioned that ...
1        A wonderful little production br br The filmin...
2        I thought this was a wonderful way to spend ti...
3        Basically theres a family where a little boy J...
4        Petter Matteis Love in the Time of Money is a ...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot bad dialogue bad acting idiotic direc...
49997    I am a Catholic taught in parochial elementary...
49998    Im going to have to disagree with the previous...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [5]:
imbd_reviews['review']=imbd_reviews['review'].str.lower()

In [6]:
imbd_reviews

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production br br the filmin...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,bad plot bad dialogue bad acting idiotic direc...,negative
49997,i am a catholic taught in parochial elementary...,negative
49998,im going to have to disagree with the previous...,negative


In [7]:
imbd_reviews=imbd_reviews.sample(frac=0.3, replace=True, random_state=1)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
vectorizer = CountVectorizer(max_features=5000)
x = vectorizer.fit_transform(imbd_reviews['review']).toarray()


In [9]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [10]:
y= imbd_reviews['sentiment']

In [11]:
from sklearn.preprocessing import LabelEncoder
lable = LabelEncoder()
y = lable.fit_transform(y)


In [12]:
y

array([1, 0, 0, ..., 0, 1, 1])

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, stratify=y)

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, fbeta_score, confusion_matrix

In [18]:
models = {
    'LR': LogisticRegression(),
    'KNN': KNeighborsClassifier(),
    'DT': DecisionTreeClassifier(),
    'RF': RandomForestClassifier(),
    'NB':GaussianNB()

}

In [19]:
for name, model in models.items():
    print(f'Model: {name}')
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    
    print('-'*120)

Model: LR


c:\Users\margen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8873333333333333
------------------------------------------------------------------------------------------------------------------------
Model: KNN
Accuracy: 0.59
------------------------------------------------------------------------------------------------------------------------
Model: DT
Accuracy: 0.7793333333333333
------------------------------------------------------------------------------------------------------------------------
Model: RF
Accuracy: 0.894
------------------------------------------------------------------------------------------------------------------------
Model: NB
Accuracy: 0.7786666666666666
------------------------------------------------------------------------------------------------------------------------


In [20]:
n_estimators = [5,20,50,100] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

# 'max_depth': max_depth,
'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}

In [21]:
randmf = RandomForestClassifier(n_estimators = 100, min_samples_split = 5, min_samples_leaf= 1, max_features = 'sqrt', max_depth= 120, bootstrap=False) 
randmf.fit( x_train, y_train) 
y_pred = randmf.predict(x_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

print('-'*120)

Accuracy: 0.89
------------------------------------------------------------------------------------------------------------------------
